### download the last month of the CDS raw forecasts 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from subprocess import call 
from shutil import which 
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser
import yaml
from yaml.loader import SafeLoader

import numpy as np
import pandas as pd
import xarray as xr

In [3]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [4]:
HOME = pathlib.Path.home()

In [5]:
CWD = pathlib.Path.cwd()

In [6]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [7]:
sys.path.append('../..')

In [8]:
from ICU_Water_Watch import C3S, domains

### parameters for papermill

In [9]:
provider = 'CDS' # should not change
varname = 'tprate' # should not change
domain_name = "C3S_download" # domain name, see below 
# if one wants to download other forecasts than the ones corresponding to the current month
lag = 0
# path where to save the hindcasts / reforecasts
gcm_path = f'/media/nicolasf/END19101/ICU/data/{provider}/operational/forecasts'
config_yaml = './CDS_config.yaml' 

### reads the yaml file, mapping GCM to system 

Note that the forecast system to system number mapping is available at: 
    
- [https://confluence.ecmwf.int/display/CKB/Description+of+the+C3S+seasonal+multi-system](https://confluence.ecmwf.int/display/CKB/Description+of+the+C3S+seasonal+multi-system)

#### it can be read directly into pandas 

In [11]:
url = 'https://confluence.ecmwf.int/display/CKB/Description+of+the+C3S+seasonal+multi-system'

In [12]:
systems_table = pd.read_html(url)

In [13]:
systems_table = systems_table[1]

In [14]:
systems_table

,Forecasting centre,Forecasting system name,CDS 'system' value
0,ECMWF,System 4,4
1,ECMWF,SEAS5,5
2,Météo-France,System 5,5
3,Météo-France,System 6,6
4,Météo-France,System 7,7
5,Météo-France,System 8,8
6,CMCC,SPS3,3
7,CMCC,SPS3.5,35
8,DWD,GCFS2.0,2
9,DWD,GCFS2.1,21


In [15]:
# Open the file and load the file
with open(config_yaml) as f:
    dict_systems = yaml.load(f, Loader=SafeLoader)

In [16]:
dict_systems

{'ECMWF': 5,
 'UKMO': 601,
 'METEO_FRANCE': 8,
 'DWD': 21,
 'CMCC': 35,
 'NCEP': 2,
 'JMA': 3,
 'ECCC_CanCM4i': 2,
 'ECCC_GEM5_NEMO': 3}

### casts the paths to `pathlib.Path` objects 

In [17]:
gcm_path = pathlib.Path(gcm_path)

In [18]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### get today's date 

In [19]:
date = dateparser.parse('today')

In [20]:
date

datetime.datetime(2022, 6, 10, 14, 59, 31, 458818)

### apply lag 

In [21]:
date = date - relativedelta(months=lag)

In [22]:
print(f"will download the forecasts issued in {date:%B %Y}")

will download the forecasts issued in June 2018


### use dask to parallelize the download of the forecasts 

#### build the queue

In [23]:
downloaded_files = []

for GCM in dict_systems.keys():

    if 'ECCC' in GCM:
        
        GCM_API_name = GCM.split('_')[0]
        
        GCM_path = gcm_path.joinpath(GCM).joinpath(varname.upper())
        
        print(f"processing ECCC, path should be {str(GCM_path)}")
    
        args = dict(GCM=GCM_API_name,
        system=dict_systems[GCM], 
        varname=varname,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=GCM_path,
        domain=domains.domains[domain_name],
        file_format='netcdf',
        level='surface',
        max_retry=3)
    
    else: 
        
        args = dict(GCM=GCM,
        system=dict_systems[GCM], 
        varname=varname,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=gcm_path.joinpath(GCM).joinpath(varname.upper()),
        domain=domains.domains[domain_name],
        file_format='netcdf',
        level='surface',
        max_retry=3)    
    
    downloaded_files.append(delayed(C3S.download)(**args))


processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_CanCM4i/TPRATE
processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_GEM5_NEMO/TPRATE


#### now compute 

In [24]:
with ProgressBar():
    downloaded_files = compute(downloaded_files)

[                                        ] | 0% Completed |  0.0s
attempting to download tprate for GCM ECCC, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM JMA, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM ECMWF, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM ECCC, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM UKMO, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM DWD, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM CMCC, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM METEO_FRANCE, for level surface, year 2018, month 6, in netcdf, attempt 1 of 3


attempting to download tprate for GCM N

2022-06-10 14:59:34,129 INFO Welcome to the CDS
2022-06-10 14:59:34,130 INFO Welcome to the CDS
2022-06-10 14:59:34,134 INFO Welcome to the CDS
2022-06-10 14:59:34,136 INFO Welcome to the CDS
2022-06-10 14:59:34,138 INFO Welcome to the CDS
2022-06-10 14:59:34,139 INFO Welcome to the CDS
2022-06-10 14:59:34,141 INFO Welcome to the CDS
2022-06-10 14:59:34,142 INFO Welcome to the CDS
2022-06-10 14:59:34,142 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,143 INFO Welcome to the CDS
2022-06-10 14:59:34,145 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,147 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels


[                                        ] | 0% Completed |  2.5s

2022-06-10 14:59:34,149 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,151 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,153 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,155 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,157 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-06-10 14:59:34,163 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels


[                                        ] | 0% Completed |  2.8s

...

[###################################     ] | 88% Completed |  3min 15.7s

2022-06-10 15:02:47,357 INFO Request is queued


[###################################     ] | 88% Completed |  3min 17.1s

2022-06-10 15:02:48,790 INFO Request is failed
2022-06-10 15:02:48,791 ERROR Message: no data is available within your requested subset
2022-06-10 15:02:48,791 ERROR Reason:  Request returned no data
2022-06-10 15:02:48,792 ERROR   Traceback (most recent call last):
2022-06-10 15:02:48,792 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2022-06-10 15:02:48,793 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2022-06-10 15:02:48,793 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2022-06-10 15:02:48,794 ERROR       result = proc(context, *context.args, **context.kwargs)
2022-06-10 15:02:48,794 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
2022-06-10 15:02:48,795 ERROR       return p(*args, **kwargs)
2022-06-10 15:02:48,795 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 60, i


failure to download or save /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_2018_06_NCEP.netcdf

[########################################] | 100% Completed |  3min 17.2s


### check dimensions and return the total number of members in the C3S MME 

In [25]:
# lfiles = downloaded_files[0]

In [26]:
# if len(lfiles) > 1: 
    
#     members = []
    
#     for f in lfiles: 
#         g = f.name.split('.')[0].split('_')[-1]
#         dset = xr.open_dataset(f)
#         dset = dset.dropna('number')
#         members.append(dset.dims['number'])
#         # print(f"{g}: {dset.dims}")
#         print(f"{g}: {dset.dims['number']}")
        
#     print(f"TOTAL NUMBER OF MEMBERS IN THE C3S MME for forecasts issued {date:%B %Y}: {np.array(members).sum()}")
    